In [0]:
!pip install gym
!pip install JSAnimation
!apt-get install python-opengl xvfb -y
!pip uninstall pyglet -y
!pip install pyglet==1.2.4
!pip install piglet
!pip install PyOpenGL
!pip install pyvirtualdisplay

# NICO2AI Reinforcement Learning I Practice

## Grid World with Q Learning
Grid Worldは2次元マトリックスで表された環境です．OpenAI GymでもFronzenLakeという環境がありますが，確率的に状態遷移する環境のため，本章ではGymと同じインターフェースをもつオリジナルの4x4マトリックスの環境を使います．この環境での目標は負の報酬が発生する穴を避けながらゴールを目指すことです．それではこの環境での強化学習設定をいかに示します．

- 状態: グリッド中の位置を示す整数値(0~15)
- 報酬: 報酬を示す整数値(0/1/-1)
- 行動: 移動する方向を示す整数値(0~3)
- 価値: 状態行動テーブルに基づくQ値

以下に環境のコードを示します．

In [0]:

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import display, clear_output

class GridWorld:
    def __init__(self):
        # s: normal space
        # h: hole
        # g: goal
        self.grid = [
            ['s', 's', 's', 's'],
            ['s', 'h', 's', 'h'],
            ['s', 's', 's', 'h'],
            ['h', 's', 's', 'g']
        ]
        self.position = 0
    
    def reset(self):
        self.position = 0
        return 0
    
    def step(self, action):
        # right
        if action == 0 and (self.position + 1) % 4 != 0:
            self.position += 1
        # left
        elif action == 1 and self.position % 4 != 0:
            self.position -= 1
        # down
        elif action == 2 and self.position < 12:
            self.position += 4
        # up
        elif action == 3 and self.position > 4:
            self.position -= 4
        obj = self.grid[int(self.position / 4)][self.position % 4]
        if obj == 's':
            reward = 0
        elif obj == 'h':
            reward = -1
        elif obj == 'g':
            reward = 1
        done = obj == 'g' or obj == 'h'
        return self.position, reward, done, self.grid

env = GridWorld()

上のコードを実行する事で前のエクササイズのバンディットと同様にGrid Worldの環境を作成できました．それではブランクを埋めてグリッドワールドのQ学習を行うエージェントを実装して見ましょう！

In [0]:
class Agent:
    def __init__(self, num_states, num_actions, discount=0.99, lr=0.1, epsilon=0.3):
        self.num_states = num_states
        self.num_actions = num_actions
        self.discount = discount
        self.lr = lr
        self.epsilon = epsilon
        self.table = np.zeros((num_states, num_actions), dtype=np.float32)
    
    def act(self, state, greedy=False):
        # epsilon-greedy action selection
        # code here
    
    def learn(self, state, action, reward, next_state, done):
        q = self.table[state][action]
        # next_q should be zero when the end of episode
        next_q = # code here
        td_error = # code here
        self.table[state][action] += # code here
        return td_error
    
    def reset(self):
        self.table = np.zeros_like(self.table, dtype=np.float32)
    
    def render_table(self):
        plt.imshow(self.table, cmap="Greens")
        ax = plt.gca()
        ax.grid(linewidth=0)
    
    def render_path(self):
        plt.imshow(np.max(self.table, axis=1).reshape(4, 4), cmap="Greens")
        ax = plt.gca()
        ax.grid(linewidth=0)

実際に作ったエージェントで学習して見ましょう！ここでは学習の様子を面白くするために，行動を行う度に発生する報酬`step_reward`を導入して，この値を変化させるとどのように学習が変化するかを観察して見ましょう．

In [0]:
# change these values later
discount = 0.9
lr = 0.1
epsilon = 0.3
step_reward = -0.01

agent = Agent(16, 4, discount, lr, epsilon)

rewards = []
for i in range(1000):
    state = env.reset()
    last_state = None
    sum_of_reward = 0
    while True:
        last_state = state
        action = agent.act(state)
        state, reward, done, _ = env.step(action)
        reward += step_reward
        agent.learn(last_state, action, reward, state, done)
        sum_of_reward += reward
        if done:
            break
    rewards.append(sum_of_reward)

In [0]:
# plot q table
agent.render_table()

In [0]:
# plot max action values
agent.render_path()

In [0]:
# plot moving average of rewards
plt.plot(np.arange(1000), np.convolve(rewards, np.ones(10) / 10, mode='same'))

予想通りの結果が得られましたか？もしそうなら割引率や学習率，ε，`step_reward`の値を変えて実験して見ましょう．

## CartPole with Q Learning
CartPoleは強化学習では有名でカートを動かしてポールを垂直に立たせるのを目標とするタスクです．最初に以下のコードを実行してアニメーションを見て見ましょう．

In [0]:
import gym
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from pyvirtualdisplay import Display

# virtual display settings to render gym
v_display = Display(visible=0, size=(1400, 900))
v_display.start()

## util function to render CartPole as GIF image
def display_frames_as_gif(frames):
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

env = gym.make('CartPole-v0')
env.reset()
frames = []
while True:
    frames.append(env.render(mode='rgb_array'))
    _, _, done, _ = env.step(0)
    if done:
        break
display_frames_as_gif(frames)

CartPoleはOpenAI Gymからも提供されています．強化学習エージェントはカートを右か左に動かします．ポールが上を向いている間，毎時刻で+1の報酬が得られます．ポールが15度以上傾くかカートが2.4（単位距離）以上移動するとエピソードが終了します．この環境の強化学習設定は以下のようになります．

- 状態: 4次元配列 (水平位置，水平速度，ポールの角度，ポールの角速度)
- 行動: カートを動かす方向を表す整数値(0/1)
- 報酬: 毎時刻+1
- 価値: テーブルで表されたQ値

GridWorldとCartPoleの違いは状態が連続値である点です．状態が連続値の場合はテーブルで扱うことができないため，状態空間を離散化する必要があります．本章では`np.linspace(start, stop, num)`と`np.digitize(x, bins)`を用いて値の離散化を行います．`np.linspace(start, stop, num)`は`start`と`end`の区間を`num`個に分割した配列を返します．`np.digitize(x, bins)`は`bins[index - 1] =< x < bins[index]`を満たす`index`を返します．それでは状態の離散化を行って見ましょう．

In [0]:
# change these values later
horizontal_position_num = 2
horizontal_velocity_num = 10
pole_angle_num = 50
pole_angular_velocity_num = 20

horizontal_position_bins = np.linspace(-2.4, 2.4, horizontal_position_num)
horizontal_velocity_bins = np.linspace(-2.0, 2.0, horizontal_velocity_num)
pole_angle_bins = np.linspace(-0.4, 0.4, pole_angle_num)
pole_angular_velocity_bins = np.linspace(-3.5, 3.5, pole_angular_velocity_num)

# return index of flatten 4 dimensional matrix
def discretize(state):
    # code here

これで，4次元の連続値を`2x10x50x20`の1次元配列のindexとして返す関数が実装できました．それでは先ほど作った強化学習エージェントで学習して見ましょう．

In [0]:
# change these values later
discount = 0.9
lr = 0.1
epsilon = 0.1

num_state = horizontal_position_num *\
        horizontal_velocity_num * pole_angle_num * pole_angular_velocity_num

agent = Agent(num_state, 2, discount, lr, epsilon)

rewards = []
for i in range(10000):
    state = env.reset()
    last_state = None
    sum_of_rewards = 0
    
    while True:
        last_state = state
        action = agent.act(discretize(state))
        state, reward, done, _ = env.step(action)
        agent.learn(discretize(last_state), action, reward, discretize(state), done)
        sum_of_rewards += reward
        if done:
            break
            
    rewards.append(sum_of_rewards)

# this may take a few minutes
plt.plot(np.arange(10000), np.convolve(rewards, np.ones(100) / 100, mode='same'))

エージェントが正しく学習されて入れば，スコアが上がっているのが確認できます．最後にエージェントの振る舞いを確認して見ましょう．

In [0]:
# evaluate trained agent
state = env.reset()
frames = []
t = 0
while True:
    frames.append(env.render(mode='rgb_array'))
    action = agent.act(discretize(state), greedy=True)
    state, _, done, _ = env.step(action)
    t += 1
    if done:
        break
env.render()
display_frames_as_gif(frames)

おめでとうございます！これでQ学習をマスターできました！